1. First we import the data into the memory



For that, I used the codes at the end of the task document

In [ ]:
! pip install -q lalsuite
! pip install -q gwpy
! pip install -q pycbc
# -- Click "restart runtime" in the runtime menu

     |████████████████████████████████| 46.2 MB 2.2 MB/s 
     |████████████████████████████████| 1.4 MB 41.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51 kB 7.3 MB/s 
     |████████████████████████████████| 295 kB 51.3 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 3.6 MB 42.3 MB/s 
     |████████████████████████████████| 1.4 MB 4.4 MB/s 
     |████████████████████████████████| 11.2 MB 43.6 MB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 6.7 MB 4.3 MB/s 
     |████████████████

In [ ]:
! pip install matplotlib==3.1.3

In [ ]:
# -- download data
! wget https://www.gw-openscience.org/s/workshop3/challenge/challenge1.gwf

--2022-02-11 14:44:05--  https://www.gw-openscience.org/s/workshop3/challenge/challenge1.gwf
Resolving www.gw-openscience.org (www.gw-openscience.org)... 131.215.113.73
Connecting to www.gw-openscience.org (www.gw-openscience.org)|131.215.113.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4025674 (3.8M)
Saving to: ‘challenge1.gwf’

challenge1.gwf      100%[===================>]   3.84M  3.00MB/s    in 1.3s    

2022-02-11 14:44:07 (3.00 MB/s) - ‘challenge1.gwf’ saved [4025674/4025674]



In [ ]:
# -- for gwpy
from gwpy.timeseries import TimeSeries
gwpy_strain = TimeSeries.read('challenge1.gwf', channel="H1:CHALLENGE1")

In [ ]:
# -- for pycbc
from pycbc import frame
pycbc_strain = frame.read_frame('challenge1.gwf', 'H1:CHALLENGE1')

Let's have a look at the data


In [ ]:
print(gwpy_strain)

Let's now determine the sampling rate of the data

In [ ]:
samplerate = gwpy_strain.sample_rate
duration = gwpy_strain.duration
# gps = gwpy_strain.t0
print('The sample rate of the data is:', samplerate, 'and the duration is: ', duration))

ModuleNotFoundError: ignored

2. We can also plot it to better visualize it

In [ ]:
# plotting in time domain using matplotlib
%matplotlib inline
plot = gwpy_strain.plot()
ax = plot.gca()
ax.set_ylabel('Amplitude')

3. We plot a spectogram/q-transform in order to identify the signal

Spectrogram:

In [ ]:
# plot spectogram
specgram = gwpy_strain.spectrogram2(fftlength=4, overlap=2, window='hann') ** (1/2.)
plot = specgram.plot()

In [ ]:
# to check for colour scale
print(specgram.min())
print(specgram.max())

In [ ]:
# adjust colour scale
ax = plot.gca()
ax.set_yscale('log')
ax.set_ylim(10, 1400)
ax.colorbar(
    clim=(1e-24, 1e-22),
    norm="log",
    label=r"Strain noise [$1/\sqrt{\mathrm{Hz}}$]",
)
plot  # refresh

Q-transform: 

By using a Q-transform, we can create a time-frequency representation of our data and see how it behaves at different frequencies over (short periods of) time, without knowing yet how the signal looks like.

In [ ]:
# time frequency representation
# maybe change frange ???
hq = gwpy_strain.q_transform(frange=(30, 500))
plot = hq.plot()
plot.colorbar(label="Normalised energy")

In [ ]:
# better visulisation
ax = plot.gca()
ax.set_epoch(64)
ax.set_ylim(30, 500)
ax.set_yscale("log")
plot  # refresh

In [ ]:
# maybe adjust qrange???
hq = gwpy_strain.q_transform(frange=(30, 500), qrange= (400, 410))
plot = hq.plot()
ax = plot.gca()
ax.set_epoch(64)
ax.set_yscale('log')
ax.colorbar(label="Normalised energy")

In [ ]:
plot.colorbars[0].mappable.set_clim(0,20)
plot.refresh()
plot

4. The time of the merger can be read off from the spectrogram and is equal to -64s.